### 파이썬 샘플코드

In [ ]:
# Python 샘플 코드
import requests

url = 'api_url'
params ={
    'serviceKey' : '서비스키', 
    'pageNo' : '1', 
    'numOfRows' : '1000', 
    'dataType' : 'XML', 
    'base_date' : '20210628', 
    'base_time' : '0600', 
    'nx' : '55', 
    'ny' : '127' 
}

response = requests.get(url, params=params)
print(response.content)

# 5. 확인한 정보들로 샘플 코드 수정
> 기본 틀 잡고 통신이 잘 되는지 확인

In [1]:
import requests

apiUrl = 'https://apis.data.go.kr/B190001/localGiftsPaymentV2/paymentsV2'
apiKeyDecoding= '디코딩 값'

params ={
    'serviceKey' : apiKeyDecoding, 
    'page' : 1, 
    'perPage' : 100000, 
    'cond[crtr_ym::GTE]' : '202201', 
    'cond[crtr_ym::LTE]' : '202201', 
    'cond[usage_rgn_cd::EQ]' : '29000', 
    'cond[par_gend::EQ]' : 'M', 
    'cond[par_ag::EQ]' : '03', 
    'cond[pvsn_inst_cd::EQ]' : 'I0000007' 
}

response = requests.get(apiUrl, params=params)
print(response.content) 

b'{"currentCount":2,"data":[{"alt_text":"9987ba4093d1ba036934038610f90b57","card_use_amt":171500,"crtr_ym":"202201","mbl_use_amt":0,"mbl_user_cnt":0,"par_ag":"03","par_gend":"M","pvsn_inst_cd":"I0000007","stlm_amt":171500,"stlm_nocs":7,"usage_rgn_cd":"29000"},{"alt_text":"bcf2e650109bc2684333d38f095beb08","card_use_amt":98500,"crtr_ym":"202201","mbl_use_amt":0,"mbl_user_cnt":0,"par_ag":"03","par_gend":"M","pvsn_inst_cd":"I0000007","stlm_amt":98500,"stlm_nocs":1,"usage_rgn_cd":"29000"}],"matchCount":51557,"page":1,"perPage":2,"totalCount":144422302}\n'


#  advanced.  원하는 데이터만 추출하는 코드 작성
> 상세사항 추가하기
>서울만,구별 필터링 해야한다. 
> 사용처지역코드로 USAGE_RGN_CD (사용처지역코드 중복 없음)	


|         지역         |   사용처지역코드    |
|:--------------------:|:--------:|
|	강북구	 |   11305	 |   
|	도봉구	  |  11320	 |    
|	노원구	  |  11350	  |   
|	중랑구	  |  11260     |  
|	동대문구	  |  11230    |   	
|	성북구	  |  11290     |  	
|	종로구	  |  11110	   |   
|	은평구	  |  11380     |  
|	서대문구	  |  11410	   |    
|	마포구	  |  11440	  | 
|	용산구	  |  11170	   | 
|	중구	      |  11140	   |   
|	성동구	  |  11200	   |
|	광진구	  |  11215	    | 
|	강동구	  |  11740	 |     
|	송파구	  |  11710	 |    
|	강남구	  |  11680	 |    
|	서초구	  |  11650	 |     
|	동작구	  |  11590	 |    
|	영등포구	  |  11560   |   
|	강서구	 |   11500	 |   
|	양천구	  |  11470	 |     
|	구로구	  |  11530	 |       
|	금천구	 |   11545	 |     
|	관악구	|    11620	 |    

### 필터링
- 이중 반복문 사용해서 외부 반복문은 구별로 내부 반복문은 만개씩 쪼갠 데이터 순서대로 저장
- 하나의 파일로 이어붙이기 

In [16]:
import requests
import json
import pandas as pd

apiEndPointUrl = 'https://apis.data.go.kr/B190001/localGiftsPaymentV2/paymentsV2'
apiKeyDecoding= '인증키'

regions = {
    '11305': '강북구', '11320': '도봉구', '11350': '노원구', '11260': '중랑구', '11230': '동대문구',
    '11290': '성북구', '11110': '종로구', '11380': '은평구', '11410': '서대문구', '11440': '마포구',
    '11170': '용산구', '11140': '중구', '11200': '성동구', '11215': '광진구', '11740': '강동구',
    '11710': '송파구', '11680': '강남구', '11650': '서초구', '11590': '동작구', '11560': '영등포구',
    '11500': '강서구', '11470': '양천구', '11530': '구로구', '11545': '금천구', '11620': '관악구'
}



for rgn_cd, rgn_name in regions.items():
    page = 1
    while 1:
        params ={
            'serviceKey' : apiKeyDecoding,
            'page' : page,
            'perPage' : 10000,
            'cond[crtr_ym::GTE]' : '201907',
            'cond[crtr_ym::LTE]' : '202308',
            'cond[usage_rgn_cd::EQ]' : rgn_cd,
            # 'cond[par_gend::EQ]' : 'M',
            # 'cond[par_ag::EQ]' : '03',
            # 'cond[pvsn_inst_cd::EQ]' : 'I0000007'
        }
    
        response = requests.get(apiEndPointUrl, params=params)
        data = response.json()
        if 'data' in data and data['data']:
            df = pd.json_normalize(data['data'])
            df.to_csv(f'{rgn_name}_{page}.csv', encoding='utf-8-sig', index=False)
            print(f"데이터를 {rgn_name}_{page}로 저장했습니다.")
            page += 1
        else:
             print("데이터 없어서 끝")
             break
    



[
    {
        "alt_text": "7c86f3585b8a0688f29c194bdb0b4830",
        "card_use_amt": 227240,
        "crtr_ym": "202201",
        "mbl_use_amt": 0,
        "mbl_user_cnt": 0,
        "par_ag": "03",
        "par_gend": "M",
        "pvsn_inst_cd": "I0000007",
        "stlm_amt": 227240,
        "stlm_nocs": 3,
        "usage_rgn_cd": "29000"
    }
]


### 1차 시도

In [1]:
# 1차 시도
# 결과
# 에러 발생 
# ChunkedEncodingError: ('Connection broken: IncompleteRead(567 bytes read, 7625 more expected)', IncompleteRead(567 bytes read, 7625 more expected))

import requests
import json
import pandas as pd

apiEndPointUrl = 'https://apis.data.go.kr/B190001/localGiftsPaymentV2/paymentsV2'
apiKeyDecoding= '인증키'

regions = {
    '11305': '강북구', '11320': '도봉구', '11350': '노원구', '11260': '중랑구', '11230': '동대문구',
    '11290': '성북구', '11110': '종로구', '11380': '은평구', '11410': '서대문구', '11440': '마포구',
    '11170': '용산구', '11140': '중구', '11200': '성동구', '11215': '광진구', '11740': '강동구',
    '11710': '송파구', '11680': '강남구', '11650': '서초구', '11590': '동작구', '11560': '영등포구',
    '11500': '강서구', '11470': '양천구', '11530': '구로구', '11545': '금천구', '11620': '관악구'
}

for rgn_cd, rgn_name in regions.items():
    page = 1
    combined_df = pd.DataFrame()
    while 1:
        params ={
            'serviceKey' : apiKeyDecoding,
            'page' : page,
            'perPage' : 10000,
            'cond[crtr_ym::GTE]' : '201907',
            'cond[crtr_ym::LTE]' : '202308',
            'cond[usage_rgn_cd::EQ]' : rgn_cd,
            # 'cond[par_gend::EQ]' : 'M',
            # 'cond[par_ag::EQ]' : '03',
            # 'cond[pvsn_inst_cd::EQ]' : 'I0000007'
        }

        response = requests.get(apiEndPointUrl, params=params)
        data = response.json()
        
        if 'data' in data and data['data']:
            df = pd.json_normalize(data['data'])
            combined_df = pd.concat([combined_df,df], ignore_index=True)
            print(f'데이터를 {rgn_name}_{page} 페이지에서 데이터를 가져왔습니다.')
            page += 1
        else:
            print(f"{rgn_name} 지역 데이터 수집 완료.")
            combined_df.to_csv(f'{rgn_name}.csv', encoding='utf-8-sig',index=False)
            break

데이터를 강북구_1 페이지에서 가져 왔습니다.
데이터를 강북구_2 페이지에서 가져 왔습니다.
데이터를 강북구_3 페이지에서 가져 왔습니다.
데이터를 강북구_4 페이지에서 가져 왔습니다.
데이터를 강북구_5 페이지에서 가져 왔습니다.
데이터를 강북구_6 페이지에서 가져 왔습니다.


ChunkedEncodingError: ('Connection broken: IncompleteRead(567 bytes read, 7625 more expected)', IncompleteRead(567 bytes read, 7625 more expected))

### 2-1차 시도 

In [ ]:
# if-else문의 else 에서 의도했던 강북구 저장이 안됨, 도봉구 저장됨

import requests
import json
import pandas as pd
from requests.exceptions import ChunkedEncodingError

apiEndPointUrl = 'https://apis.data.go.kr/B190001/localGiftsPaymentV2/paymentsV2'
apiKeyDecoding= '인증키'

regions = {
    '11305': '강북구', '11320': '도봉구', '11350': '노원구', '11260': '중랑구', '11230': '동대문구',
    '11290': '성북구', '11110': '종로구', '11380': '은평구', '11410': '서대문구', '11440': '마포구',
    '11170': '용산구', '11140': '중구', '11200': '성동구', '11215': '광진구', '11740': '강동구',
    '11710': '송파구', '11680': '강남구', '11650': '서초구', '11590': '동작구', '11560': '영등포구',
    '11500': '강서구', '11470': '양천구', '11530': '구로구', '11545': '금천구', '11620': '관악구'
}

for rgn_cd, rgn_name in regions.items():
    page = 1
    combined_df = pd.DataFrame()
    while True:
        params ={
            'serviceKey' : apiKeyDecoding,
            'page' : page,
            'perPage' : 10000,
            'cond[crtr_ym::GTE]' : '201907',
            'cond[crtr_ym::LTE]' : '202308',
            'cond[usage_rgn_cd::EQ]' : rgn_cd,
        }

        try:
            response = requests.get(apiEndPointUrl, params=params)
            response.raise_for_status()
            data = response.json()
        except ChunkedEncodingError as ce:
            print(f'ChunkedEncodingError 발생. 재시도합니다. 에러 메시지: {ce}')
            continue
        except requests.exceptions.RequestException as re:
            print(f'요청 중 오류 발생: {re}')
            break

        if 'data' in data and data['data']:
            df = pd.json_normalize(data['data'])
            combined_df = pd.concat([combined_df,df], ignore_index=True)
            print(f'데이터를 {rgn_name}_{page} 페이지에서 가져왔습니다.')
            page += 1
        else:
            print(f"{rgn_name} 지역 데이터 수집 완료.")
            combined_df.to_csv(f'{rgn_name}.csv', encoding='utf-8-sig', index=False)
            break


### 2-2시도

In [ ]:
# 2-2차 시도 
# ChunkedEncodingError(http통신 중 발생하는 일반적인 오류, 데이터가 서버에서 예상보다 큰 Chunk 덩어리로 전송되거나 끊김 발생)와 같은 특정 예외를 처리하고
# HTTP 재시도 메커니즘을 사용하여 오류에 대한 대응을 시도합니다.
# 결과
# 강북구, 도봉구 저장, 
# 노원구, 종로구, 서대문구에서 종료 발생

import requests
import json
import pandas as pd
from requests.exceptions import ChunkedEncodingError

apiEndPointUrl = 'https://apis.data.go.kr/B190001/localGiftsPaymentV2/paymentsV2'
apiKeyDecoding= '인증키'

regions = {
    '11305': '강북구', '11320': '도봉구', '11350': '노원구', '11260': '중랑구', '11230': '동대문구',
    '11290': '성북구', '11110': '종로구', '11380': '은평구', '11410': '서대문구', '11440': '마포구',
    '11170': '용산구', '11140': '중구', '11200': '성동구', '11215': '광진구', '11740': '강동구',
    '11710': '송파구', '11680': '강남구', '11650': '서초구', '11590': '동작구', '11560': '영등포구',
    '11500': '강서구', '11470': '양천구', '11530': '구로구', '11545': '금천구', '11620': '관악구'
}

for rgn_cd, rgn_name in regions.items():
    page = 1
    combined_df = pd.DataFrame()
    while True:
        try:
            params = {
                'serviceKey': apiKeyDecoding,
                'page': page,
                'perPage': 10000,
                'cond[crtr_ym::GTE]': '201907',
                'cond[crtr_ym::LTE]': '202308',
                'cond[usage_rgn_cd::EQ]': rgn_cd,
            }

            response = requests.get(apiEndPointUrl, params=params)
            response.raise_for_status()
            data = response.json()

            if 'data' in data and data['data']:
                df = pd.json_normalize(data['data'])
                combined_df = pd.concat([combined_df, df], ignore_index=True)
                print(f'데이터를 {rgn_name}_{page} 페이지에서 데이터를 가져왔습니다.')
                page += 1
            else:
                print(f"{rgn_name} 지역 데이터 수집 완료.")
                combined_df.to_csv(f'{rgn_name}.csv', encoding='utf-8-sig', index=False)
                break

        except ChunkedEncodingError as ce:
            print(f'ChunkedEncodingError 발생. 재시도합니다. 에러 메시지: {ce}')
            continue
        except requests.exceptions.RequestException as re:
            print(f'요청 중 오류 발생: {re}')
            break


데이터를 강북구_1 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_2 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_3 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_4 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_5 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_6 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_7 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_8 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_9 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_10 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_11 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_12 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_13 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_14 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_15 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_16 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_17 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_18 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_19 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_20 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_21 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_22 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_23 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_24 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_25 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_26 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_27 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_28 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_29 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_30 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_31 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_32 페이지에서 데이터를 가져왔습니다.
데이터를 강북구_33 페이지에서

### 최종

In [1]:
# 3차 시도
# Retry Mechanism (재시도 메커니즘) : https요청 실패한 경우 계속 재시도, request 라이브러리의 requests.exceptions.RequestException으로 로직 구현
# Backoff Strategy (지수 백오프 전략) : 재시도 시점을 조절, 초기에는 빠르게 재시도 일정 횟수 이상 실패한 경우 재시도 간격을 늘림(서버 과부하 방지)

import requests
import json
import pandas as pd

#재시도 로직 구현 라이브러리
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


apiEndPointUrl = 'https://apis.data.go.kr/B190001/localGiftsPaymentV2/paymentsV2'
apiKeyDecoding= '인증키'

regions = {
    '11305': '강북구', '11320': '도봉구', '11350': '노원구', '11260': '중랑구', '11230': '동대문구',
    '11290': '성북구', '11110': '종로구', '11380': '은평구', '11410': '서대문구', '11440': '마포구',
    '11170': '용산구', '11140': '중구', '11200': '성동구', '11215': '광진구', '11740': '강동구',
    '11710': '송파구', '11680': '강남구', '11650': '서초구', '11590': '동작구', '11560': '영등포구',
    '11500': '강서구', '11470': '양천구', '11530': '구로구', '11545': '금천구', '11620': '관악구'
}

for rgn_cd, rgn_name in regions.items():
    page = 1
    combined_df = pd.DataFrame()

    while True:
        try:
            params = {
                'serviceKey': apiKeyDecoding,
                'page': page,
                'perPage': 10000,
                'cond[crtr_ym::GTE]': '201907',
                'cond[crtr_ym::LTE]': '202308',
                'cond[usage_rgn_cd::EQ]': rgn_cd,
            }

            # 재시도 로직
            session = requests.Session() # HTTP 요청을 보낼때 세션 생성(세션, 연결 유지 & 재사용 → 성능 개선됨 )
            retry = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
                         # 5번까지 재시도,  재시도 간격,  특정 HTTP 상태코드가 발생할 때만 재시도
            adapter = HTTPAdapter(max_retries=retry) # 위에서 설정한 재시도 옵션 재사용
            session.mount('http://', adapter) # HTTP 요청중 http://프로토콜 사용하는 경우 어댑터를 세션에 연결 
            session.mount('https://', adapter) # HTTP 요청중 https://프로토콜 사용하는 경우 어댑터를 세션에 연결

            # API 요청 보내기
            response = session.get(apiEndPointUrl, params=params)
            response.raise_for_status() # HTTP 오류가 발생하면 예외시킴
            data = response.json()

            # 데이터 프레임으로 만들고 이어붙인뒤 저장
            if 'data' in data and data['data']:
                df = pd.json_normalize(data['data'])
                combined_df = pd.concat([combined_df, df], ignore_index=True)
                print(f"데이터를 {rgn_name}_{page} 페이지에서 가져옵니다.")
                page += 1
            else:
                print(f"{rgn_name} 지역 데이터 수집 완료.")
                combined_df.to_csv(f'{rgn_name}.csv', encoding='utf-8-sig', index=False)
                break
        except requests.exceptions.RequestException as re:
            print(f"오류 발생: {re}")
            continue
            

데이터를 노원구_1 페이지에서 가져옵니다.
데이터를 노원구_2 페이지에서 가져옵니다.
데이터를 노원구_3 페이지에서 가져옵니다.
데이터를 노원구_4 페이지에서 가져옵니다.
데이터를 노원구_5 페이지에서 가져옵니다.
데이터를 노원구_6 페이지에서 가져옵니다.
데이터를 노원구_7 페이지에서 가져옵니다.
데이터를 노원구_8 페이지에서 가져옵니다.
데이터를 노원구_9 페이지에서 가져옵니다.
데이터를 노원구_10 페이지에서 가져옵니다.
데이터를 노원구_11 페이지에서 가져옵니다.
데이터를 노원구_12 페이지에서 가져옵니다.
데이터를 노원구_13 페이지에서 가져옵니다.
데이터를 노원구_14 페이지에서 가져옵니다.
데이터를 노원구_15 페이지에서 가져옵니다.
데이터를 노원구_16 페이지에서 가져옵니다.
데이터를 노원구_17 페이지에서 가져옵니다.
데이터를 노원구_18 페이지에서 가져옵니다.
데이터를 노원구_19 페이지에서 가져옵니다.
데이터를 노원구_20 페이지에서 가져옵니다.
데이터를 노원구_21 페이지에서 가져옵니다.
데이터를 노원구_22 페이지에서 가져옵니다.
데이터를 노원구_23 페이지에서 가져옵니다.
데이터를 노원구_24 페이지에서 가져옵니다.
데이터를 노원구_25 페이지에서 가져옵니다.
데이터를 노원구_26 페이지에서 가져옵니다.
데이터를 노원구_27 페이지에서 가져옵니다.
데이터를 노원구_28 페이지에서 가져옵니다.
데이터를 노원구_29 페이지에서 가져옵니다.
데이터를 노원구_30 페이지에서 가져옵니다.
데이터를 노원구_31 페이지에서 가져옵니다.
데이터를 노원구_32 페이지에서 가져옵니다.
데이터를 노원구_33 페이지에서 가져옵니다.
데이터를 노원구_34 페이지에서 가져옵니다.
데이터를 노원구_35 페이지에서 가져옵니다.
데이터를 노원구_36 페이지에서 가져옵니다.
오류 발생: ('Connection broken: IncompleteRead(567 bytes read, 7625 more expected)', IncompleteRead(567 bytes rea